# Text Classification Using The LSTM Deep Learning Model

## Import necesarry Libraries

In [16]:
!pip install sastrawi pandas numpy tensorflow scikit-learn

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pickle
import re
import os

## Define Categories

In [18]:
ALLOWED_CATEGORIES = [
    'Allowance',
    'Asset Sale',
    'Bonus',
    'Dividend',
    'Freelance',
    'Gift',
    'Holiday Bonus',
    'Inheritance',
    'Investment',
    'Other',
    'Pension',
    'Refund',
    'Rental',
    'Salary',
    'Small Business'
]

## Deleting Model Artifacts

In [19]:
artifacts = ['model_artifacts/finance_model.h5', 'model_artifacts/tokenizer.pkl', 'model_artifacts/label_mappings.pkl']

for file in artifacts:
    if os.path.exists(file):
        os.remove(file)
        print(f"Removed previous {file}")

Removed previous model_artifacts/finance_model.h5
Removed previous model_artifacts/tokenizer.pkl
Removed previous model_artifacts/label_mappings.pkl


## Text Preprocessing

In [20]:
factory = StemmerFactory()
lemmatizer = factory.create_stemmer()

# Enhanced financial terms dictionary with English-Indonesian mappings
FINANCIAL_TERMS = {
    # Payment methods
    "gopay": "gopay", "ovo": "ovo", "dana": "dana", "shopeepay": "shopeepay",
    # Banks
    "bca": "bca", "bni": "bni", "bri": "bri", "mandiri": "mandiri",
    # Financial terms
    "kpr": "kpr", "atm": "atm", "rekening": "rekening", "deposito": "deposito",
    # Providers
    "pln": "pln", "pdam": "pdam", "telkomsel": "telkomsel", "indihome": "indihome",
    # Currencies
    "rp": "rp", "juta": "juta", "ribu": "ribu",
    # Modern terms
    "bibit": "investasi", "pluang": "investasi", "stockbit": "investasi",
    "fitness": "gym", "center": "gym", "membership": "member",
    "top up": "isi ulang", "invest": "investasi", "saham": "investasi",
    # English terms mapped to Indonesian
    "salary": "gaji", "wage": "gaji", "income": "pendapatan",
    "bonus": "bonus", "dividend": "dividen", "freelance": "pekerja lepas",
    "gift": "hadiah", "present": "hadiah", "allowance": "uang saku",
    "pension": "pensiun", "refund": "pengembalian dana",
    "rent": "sewa", "rental": "sewa", "investment": "investasi",
    "business": "usaha", "sale": "penjualan", "asset": "aset",
    "inheritance": "warisan", "holiday": "liburan", "thr": "tunjangan hari raya"
}

def preprocess_text(text):
    """Enhanced bilingual preprocessing with special handling for financial terms"""
    # Special handling for common mis-stemmed terms
    preservation_terms = {
        'warisan': 'warisan',
        'dividens': 'dividen',
        'returns': 'return',
        'investasiasi': 'investasi',
        'investasiment': 'investasi',
        'dividend': 'dividen',
        'roi': 'return investasi'
    }

    # Convert to lowercase
    text = text.lower()

    # Replace preserved terms before any processing
    for term, replacement in preservation_terms.items():
        text = text.replace(term, replacement)

    # Replace financial terms (both English and Indonesian)
    for term, replacement in FINANCIAL_TERMS.items():
        text = text.replace(term, replacement)

    # Remove special chars but keep basic punctuation
    text = re.sub(r'[^\w\s.,]', '', text)

    # Conservative stemming
    tokens = []
    for token in text.split():
        if token in preservation_terms:
            tokens.append(preservation_terms[token])
        elif token in FINANCIAL_TERMS:
            tokens.append(FINANCIAL_TERMS[token])
        else:
            stemmed = lemmatizer.stem(token)
            tokens.append(stemmed)

    return ' '.join(tokens)

## Loading Data

In [21]:
def load_and_augment_data(csv_path):
    """Load data with bilingual augmentation"""
    df = pd.read_csv(csv_path)

    # Filter to only include allowed categories
    df = df[df['label'].isin(ALLOWED_CATEGORIES)]

    # Add bilingual transaction examples
    bilingual_transactions = [
        ["Gaji bulanan dari perusahaan", "Salary"],
        ["Monthly salary from company", "Salary"],
        ["Bonus kinerja triwulan", "Bonus"],
        ["Performance bonus for Q3", "Bonus"],
        ["Pendapatan freelance desain", "Freelance"],
        ["Freelance UI design income", "Freelance"],
        ["Dividen saham dari investasi", "Dividend"],
        ["Stock dividend from investments", "Dividend"],
        ["Pendapatan sewa apartemen", "Rental"],
        ["Apartment rental income", "Rental"],
        ["Uang THR lebaran dari kantor", "Holiday Bonus"],
        ["Year-end holiday bonus", "Holiday Bonus"],
        ["Penjualan motor bekas", "Asset Sale"],
        ["Used motorcycle sale", "Asset Sale"],
        ["Uang saku bulanan dari orang tua", "Allowance"],
        ["Monthly allowance from parents", "Allowance"],
        ["Warisan dari kakek", "Inheritance"],
        ["Inheritance from grandfather", "Inheritance"],
        ["Pensiun bulanan dari BPJS", "Pension"],
        ["Monthly pension from BPJS", "Pension"],
        ["Refund tiket pesawat", "Refund"],
        ["Flight ticket refund", "Refund"],
        ["Pendapatan usaha kafe", "Small Business"],
        ["Cafe business income", "Small Business"],
        ["Hadiah ulang tahun", "Gift"],
        ["Birthday present", "Gift"],
        ["Keuntungan investasi saham", "Investment"],
        ["Stock investment profit", "Investment"],
        ["Pembagian dividen saham teknologi", "Dividend"],
        ["Dividend from tech stocks", "Dividend"],
        ["Return on investment portfolio", "Investment"],
        ["Keuntungan portofolio investasi", "Investment"],
        ["Warisan orang tua berupa properti", "Inheritance"],
        ["Inheritance of family property", "Inheritance"],
        ["Uang jajan anak sekolah", "Allowance"],
        ["Duit saku bulanan anak", "Allowance"],
        ["Pendapatan toko kelontong", "Small Business"],
        ["Revenue from corner shop", "Small Business"],
        ["Hadiah anniversary pernikahan", "Gift"],
        ["Bingkisan ulang tahun", "Gift"]
    ]

    modern_df = pd.DataFrame(bilingual_transactions, columns=['text', 'label'])
    df = pd.concat([df, modern_df])

    # Filter again to ensure no invalid categories slipped in
    df = df[df['label'].isin(ALLOWED_CATEGORIES)]

    # Preprocess all text
    df['processed_text'] = df['text'].apply(preprocess_text)
    return df

## Model Definition

In [22]:
def create_lstm_model(vocab_size, num_classes, max_len):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=256, mask_zero=True),

        # Both BiLSTMs return sequences
        Bidirectional(LSTM(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2)),
        Bidirectional(LSTM(64, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2)),

        # Now GlobalMaxPooling will work
        GlobalMaxPooling1D(),

        Dense(128, activation='relu', kernel_regularizer='l2'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
        loss='categorical_crossentropy',
        metrics=['accuracy',
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')]
    )
    return model

## Model Training

In [23]:
def train_model(df, model_save_path='model_artifacts/finance_model.h5'):
    # Prepare labels - only using our allowed categories
    label2id = {label: i for i, label in enumerate(ALLOWED_CATEGORIES)}
    id2label = {i: label for i, label in enumerate(ALLOWED_CATEGORIES)}

    # Filter out any labels not in our allowed categories (just in case)
    df = df[df['label'].isin(ALLOWED_CATEGORIES)]
    y = pd.get_dummies(df['label'].map(label2id)).values

    # Tokenization
    tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>", filters='')
    tokenizer.fit_on_texts(df['processed_text'])
    vocab_size = len(tokenizer.word_index) + 1

    # Sequence preparation
    sequences = tokenizer.texts_to_sequences(df['processed_text'])
    max_len = max(len(seq) for seq in sequences)
    X = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

    # Class weights
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(df['label'].map(label2id)),
        y=df['label'].map(label2id)
    )
    class_weights = dict(enumerate(class_weights))

    # Train-test split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=df['label']
    )

    # Model training
    model = create_lstm_model(vocab_size, len(ALLOWED_CATEGORIES), max_len)

    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint(model_save_path, save_best_only=True)
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        class_weight=class_weights,
        callbacks=callbacks
    )

    # Save artifacts
    with open('model_artifacts/tokenizer.pkl', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('model_artifacts/label_mappings.pkl', 'wb') as handle:
        pickle.dump((label2id, id2label), handle, protocol=pickle.HIGHEST_PROTOCOL)

    return model, tokenizer, label2id, id2label, max_len


## Model Evaluation and Testing

In [24]:
class FinancialClassifier:
    def __init__(self, model_path='model_artifacts/finance_model.h5'):
        self.model = load_model(model_path)
        with open('model_artifacts/tokenizer.pkl', 'rb') as handle:
            self.tokenizer = pickle.load(handle)
        with open('model_artifacts/label_mappings.pkl', 'rb') as handle:
            self.label2id, self.id2label = pickle.load(handle)
        self.max_len = self.model.input_shape[1]

    def predict(self, text, confidence_threshold=0.7):
        # Enhanced bilingual preprocessing
        processed = preprocess_text(text)
        seq = self.tokenizer.texts_to_sequences([processed])
        padded = pad_sequences(seq, maxlen=self.max_len, padding='post', truncating='post')

        # Predict
        proba = self.model.predict(padded, verbose=0)[0]
        pred_id = np.argmax(proba)
        confidence = proba[pred_id]

        # Prepare results
        result = {
            'original_text': text,
            'processed_text': processed,
            'prediction': self.id2label[pred_id],
            'confidence': float(confidence),
            'all_predictions': {
                self.id2label[i]: float(p)
                for i, p in enumerate(proba)
                if p > 0.05
            }
        }

        # Apply confidence threshold
        if confidence < confidence_threshold:
            result['prediction'] = 'Other'  # Default to 'Other' for low confidence
            result['suggestion'] = 'Low confidence prediction'

        return result

In [25]:
if __name__ == "__main__":
    try:
        # 1. Load and augment data
        df = load_and_augment_data('datasets/transactions.csv')
        print(f"Loaded {len(df)} samples with {len(df['label'].unique())} categories")
        print("Categories:", df['label'].unique())

        # 2. Train the model
        print("\nTraining model...")
        model, tokenizer, label2id, id2label, max_len = train_model(df)

        # 3. Initialize classifier
        classifier = FinancialClassifier()

        # 4. Test predictions with bilingual examples
        test_cases = [
            "Gaji bulan Desember dari kantor",
            "Monthly salary from employer",
            "Bonus kinerja tahunan",
            "Annual performance bonus",
            "Pendapatan freelance programming",
            "Freelance writing income",
            "Dividen saham teknologi",
            "Tech stock dividends",
            "Pendapatan sewa rumah",
            "House rental income",
            "Uang THR lebaran",
            "Year-end holiday allowance",
            "Penjualan laptop bekas",
            "Used car sale proceeds",
            "Uang saku mingguan",
            "Weekly allowance money",
            "Warisan keluarga",
            "Family inheritance",
            "Pensiun bulanan",
            "Monthly pension payment",
            "Pengembalian dana tiket",
            "Concert ticket refund",
            "Pendapatan usaha kecil",
            "Small business revenue",
            "Hadiah pernikahan",
            "Wedding gift money",
            "Keuntungan investasi",
            "Investment returns"
        ]

        print("\nBilingual Test Predictions:")
        for text in test_cases:
            result = classifier.predict(text)
            print(f"\nOriginal: {result['original_text']}")
            print(f"Processed: {result['processed_text']}")
            print(f"Prediction: {result['prediction']} (Confidence: {result['confidence']:.2%})")
            if result['prediction'] == 'Other':
                print(f"⚠️ Low confidence - categorized as Other")
            print("Details:")
            for cat, prob in sorted(result['all_predictions'].items(), key=lambda x: x[1], reverse=True):
                if prob > 0.05:
                    print(f"- {cat}: {prob:.2%}")

    except FileNotFoundError:
        print("Error: File 'transactions.csv' not found")
    except Exception as e:
        print(f"Error: {str(e)}")

Loaded 216 samples with 15 categories
Categories: ['Salary' 'Bonus' 'Freelance' 'Dividend' 'Rental' 'Holiday Bonus'
 'Asset Sale' 'Allowance' 'Inheritance' 'Pension' 'Refund'
 'Small Business' 'Gift' 'Investment' 'Other']

Training model...
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'global_max_pooling1d_2' (of type GlobalMaxPooling1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.0214 - loss: 3.9564 - precision: 0.0000e+00 - recall: 0.0000e+00

6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 520ms/step - accuracy: 0.0250 - loss: 3.9553 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2045 - val_loss: 3.8953 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.1480 - loss: 3.8365 - precision: 0.0000e+00 - recall: 0.0000e+00

6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 149ms/step - accuracy: 0.1509 - loss: 3.8419 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.4091 - val_loss: 3.8226 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.3510 - loss: 3.8330 - precision: 0.0000e+00 - recall: 0.0000e+00

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - accuracy: 1.0000 - loss: 0.4211 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.8864 - val_loss: 0.8189 - val_precision: 0.9459 - val_recall: 0.7955
Epoch 38/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 1.0000 - loss: 0.4281 - precision: 1.0000 - recall: 1.0000

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 1.0000 - loss: 0.4267 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.8864 - val_loss: 0.7968 - val_precision: 0.9250 - val_recall: 0.8409
Epoch 39/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 1.0000 - loss: 0.3895 - precision: 1.0000 - recall: 1.0000

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 1.0000 - loss: 0.3899 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.8864 - val_loss: 0.7846 - val_precision: 0.9250 - val_recall: 0.8409
Epoch 40/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 1.0000 - loss: 0.3929 - precision: 1.0000 - recall: 1.0000

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 229ms/step - accuracy: 1.0000 - loss: 0.3922 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.8636 - val_loss: 0.7829 - val_precision: 0.9024 - val_recall: 0.8409
Epoch 41/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 233ms/step - accuracy: 1.0000 - loss: 0.3739 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.8636 - val_loss: 0.7883 - val_precision: 0.9000 - val_recall: 0.8182
Epoch 42/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step - accuracy: 1.0000 - loss: 0.3707 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.8864 - val_loss: 0.7970 - val_precision: 0.8780 - val_recall: 0.8182
Epoch 43/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - accuracy: 1.0000 - loss: 0.3551 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.8636 - val_loss: 0.7884 - val_precision: 0.8780 - val_recall: 0.8182
Epoch 44/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 1.0000 - loss: 0.3485 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.8864 - val_loss: 0.7993 - val_precis


Bilingual Test Predictions:

Original: Gaji bulan Desember dari kantor
Processed: gaji bulan desember dari kantor
Prediction: Salary (Confidence: 94.58%)
Details:
- Salary: 94.58%

Original: Monthly salary from employer
Processed: monthly gaji from employer
Prediction: Salary (Confidence: 97.89%)
Details:
- Salary: 97.89%

Original: Bonus kinerja tahunan
Processed: bonus kerja tahun
Prediction: Bonus (Confidence: 97.45%)
Details:
- Bonus: 97.45%

Original: Annual performance bonus
Processed: annual performance bonus
Prediction: Bonus (Confidence: 98.53%)
Details:
- Bonus: 98.53%

Original: Pendapatan freelance programming
Processed: dapat kerja lepas programming
Prediction: Freelance (Confidence: 97.73%)
Details:
- Freelance: 97.73%

Original: Freelance writing income
Processed: kerja lepas writing dapat
Prediction: Freelance (Confidence: 97.91%)
Details:
- Freelance: 97.91%

Original: Dividen saham teknologi
Processed: dividen investasi teknologi
Prediction: Dividend (Confidence: 95.